In this I'm going to test BERTopic for topic modeling. It seems to have significant advantages over LDA.

Going to just follow a tutorial and use their dataset for now

In [40]:
from bertopic import BERTopic
import pandas as pd
import pickle

In [66]:
# Since BERT uses sentence embeddings in it's process we want the stop words but the removed ones are useless
# Testing on this one but it's really small so idk if the topics will be good
print(f"Reading in data")
dfCs = []
dfCs.append(pd.read_json("C:\\Users\\rharter2\\Documents\\Git\\Optus_data\\reddit\\opiates_comments1.json"))
dfCs.append(pd.read_json("C:\\Users\\rharter2\\Documents\\Git\\Optus_data\\reddit\\opiates_comments2.json"))
dfCs.append(pd.read_json("C:\\Users\\rharter2\\Documents\\Git\\Optus_data\\reddit\\opiates_comments3.json"))
dfCs.append(pd.read_json("C:\\Users\\rharter2\\Documents\\Git\\Optus_data\\reddit\\opiates_comments4.json"))
dfCs.append(pd.read_json("C:\\Users\\rharter2\\Documents\\Git\\Optus_data\\reddit\\opiates_comments5.json"))
dfCs.append(pd.read_json("C:\\Users\\rharter2\\Documents\\Git\\Optus_data\\reddit\\opiates_comments6.json"))
dfCs.append(pd.read_json("C:\\Users\\rharter2\\Documents\\Git\\Optus_data\\reddit\\opiates_comments7.json"))

for i in range(len(dfCs)):
    print(f"File {i} shape: {dfCs[i].shape}")



Reading in data
File 0 shape: (1104, 23)
File 1 shape: (206111, 24)
File 2 shape: (628170, 23)
File 3 shape: (934186, 27)
File 4 shape: (1235399, 54)
File 5 shape: (1014397, 56)
File 6 shape: (58013, 51)


In [67]:
print(f"Combining data")
dfC = pd.concat(dfCs, ignore_index=True)
# dfC = pd.concat([dfC, pd.read_json("C:\\Users\\rharter2\\Documents\\Git\\Optus_data\\reddit\\opiates_comments2.json")])
# dfC = pd.concat([dfC, pd.read_json("C:\\Users\\rharter2\\Documents\\Git\\Optus_data\\reddit\\opiates_comments3.json")])
# dfC = pd.concat([dfC, pd.read_json("C:\\Users\\rharter2\\Documents\\Git\\Optus_data\\reddit\\opiates_comments4.json")])
# dfC = pd.concat([dfC, pd.read_json("C:\\Users\\rharter2\\Documents\\Git\\Optus_data\\reddit\\opiates_comments5.json")])
# dfC = pd.concat([dfC, pd.read_json("C:\\Users\\rharter2\\Documents\\Git\\Optus_data\\reddit\\opiates_comments6.json")])
# dfC = pd.concat([dfC, pd.read_json("C:\\Users\\rharter2\\Documents\\Git\\Optus_data\\reddit\\opiates_comments7.json")])
print(f"Shape is {dfC.shape}")

Combining data
Shape is (4077380, 63)


In [68]:
print(f"Munging data")

dfC = dfC.loc[(dfC['body'] != '') & (dfC['body'] != '[deleted]') & (dfC['body'] != '[removed]')]
dfC.reset_index(drop=True, inplace=True)

print(f"Shape is {dfC.shape}")


Munging data
Shape is (3878974, 63)


In [37]:
print("sampling data")
dfSample = dfC['body'].sample(1000)
data = dfSample.to_list()

display(dfSample)
len(data)

sampling data


1134883    That we are!! It's always nice to find people ...
773476     I don't use foil. I smoke the tar off of the p...
1658265    Brazil..yes its weaker than Tramadol its 5 tim...
13618      someone who isn't me. It does nothing as far a...
2182275    wow, we get people from literally all over the...
                                 ...                        
3225307    I still feel like this could be an alternate u...
286240     I've never iv'ed opana but I prefer snorted op...
2688987    if you've been on subs for a long period of ti...
1486657    damn whole weeks paycheck? how much do you get...
3422126    where i was living 4 years ago thats whag my d...
Name: body, Length: 1000, dtype: object

1000

In [38]:
with open("data.pkl", "wb") as fOut:
    pickle.dump({'data': data},fOut)


In [39]:
with open("data.pkl", "rb") as fIn:
    cache_data = pickle.load(fIn)
    data = cache_data['data']
    display(len(data))

1000

In [46]:
# precalculate sentence embeddings so they don't have to be done each time and save them
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("all-MiniLM-L6-v2") # fast but pretty accurate embeddings
embeddings = embedding_model.encode(data, show_progress_bar=True)

with open("embeddings.pkl", "wb") as fOut:
    pickle.dump({'embeddings-only1': embeddings},fOut)



Batches: 100%|██████████| 32/32 [00:13<00:00,  2.33it/s]


In [ ]:
# Load embeddings from file
with open("embeddings.pkl", "rb") as fIn:
    cache_data = pickle.load(fIn)
    embeddings = cache_data['embeddings']

In [47]:
# Default representations. Want to try another representation that has less stopwords
topicModel = BERTopic(verbose=True, embedding_model="all-MiniLM-L6-v2") 
topics, probs = topicModel.fit_transform(data, embeddings)

2023-10-31 16:50:17,688 - BERTopic - Reduced dimensionality
2023-10-31 16:50:17,747 - BERTopic - Clustered reduced embeddings


In [62]:
# A maybe better representation?
from bertopic.representation import KeyBERTInspired

topicModel = BERTopic(verbose=True, embedding_model="all-MiniLM-L6-v2", representation_model=KeyBERTInspired(), calculate_probabilities=True, nr_topics=8) 
topics, probs = topicModel.fit_transform(data, embeddings)

2023-10-31 17:02:16,901 - BERTopic - Reduced dimensionality
2023-10-31 17:02:17,001 - BERTopic - Clustered reduced embeddings
2023-10-31 17:02:25,371 - BERTopic - Reduced number of topics from 15 to 8


In [ ]:
topicModel.save("keyBertModelReddit-model", serialization='safetensors')

In [41]:
topicModel = BERTopic(verbose=True) 
topicModel.load('keyBertModelReddit-model')

In [63]:
topicModel.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,456,-1_pain_will_take_had,"[pain, will, take, had, can, get, getting, hav...",[I was on vivitrol and it works very well as l...
1,0,220,0_opiates_heroin_drugs_pills,"[opiates, heroin, drugs, pills, oxy, get, take...",[If I interpreted one of your other comments c...
2,1,166,1_writing_read_reading_haha,"[writing, read, reading, haha, hahaha, lol, fe...",[I always felt so. They are incredibly intelli...
3,2,78,2_dealers_dealer_vendor_buy,"[dealers, dealer, vendor, buy, sell, prices, s...",[Thanks man. This one is a little over an Oz (...
4,3,30,3_cop_restraining_cops_police,"[cop, restraining, cops, police, call, lol, yo...","[I mean, it's ultimately not really your fault..."
5,4,21,4_butthurt_idiot_stupid_quote,"[butthurt, idiot, stupid, quote, read, say, sa...",[That has nothing to do with what I just said....
6,5,16,5_vein_veins_needles_needle,"[vein, veins, needles, needle, abscess, blood,...",[Break through the skin and if the vein rolls ...
7,6,13,6_subs_sub_block_stop,"[subs, sub, block, stop, for, honestly, about,...",[he just mentioned he was on subs and is only...


In [53]:
topicModel.get_topic(0)

[('writing', 0.40805572),
 ('reading', 0.30582643),
 ('read', 0.29275584),
 ('thank', 0.26279113),
 ('thanks', 0.25379178),
 ('listened', 0.2475191),
 ('much', 0.24146655),
 ('made', 0.22903267),
 ('love', 0.20906344),
 ('all', 0.20592237)]

In [64]:
# Want to get this working
topicModel.visualize_distribution(topicModel.probabilities_[0])

In [65]:
topicModel.visualize_topics()